<a href="https://colab.research.google.com/github/MattFleisch/AI_1/blob/main/AI2_Tumour_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Brain Tumour Classification**
---
### Ariel Levy (LVYARI002)
### Matthew Flesichman (FLSMAT002)
---
##### This convolutional neural network (CNN) makes use of a [dataset](https://www.kaggle.com/datasets/sartajbhuvaji/brain-tumor-classification-mri) of 3264 labelled brain scan images to predict the presence and type of tumours. There are four possible classes: *meningioma tumour*, *glioma tumour*, *pituitary tumour*, and *no tumour*. The model achieves an ~90% testing accuracy, signifcantly higher than that of random chance (25%).
---

## Libraries
These are the libraries that need to be imported in order to construct and display the results of the model.


In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense
import tensorflow.keras.backend as K
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import optimizers
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split

## Data Extraction
Here the .zip file is extracted to a folder and its subdirectories are determined. If you download the dataset make sure the file is named `tumor.zip`.

In [2]:
from zipfile import ZipFile
filename = "tumor"

with ZipFile(f'{filename}.zip', 'r') as zipObj:
   zipObj.extractall(f'{filename}')

import glob
f1 = glob.glob(f'/content/{filename}/*')
f1 = [f for f in f1 if '__MACOSX' not in f]
print(f1)

FileNotFoundError: [Errno 2] No such file or directory: 'tumor.zip'

## Data Preparation
Here the image files (along with their labels) are moved from folders into arrays; training and testing data is combined so that the split can be easily changed later. Image files are converted to greyscale and 64x64, making analysis easier and more resource efficient.

In [ ]:
import os
import cv2
import numpy as np

images = []
labels = []
image_size = 64

for dataset_path in f1:
    # Get the class names from the subdirectories within 'Training' and 'Testing'
    class_names = [name for name in os.listdir(dataset_path) if os.path.isdir(os.path.join(dataset_path, name))]

    for class_index, class_name in enumerate(class_names):
        class_path = os.path.join(dataset_path, class_name)
        if os.path.exists(class_path):
            for image_name in os.listdir(class_path):
                image_path = os.path.join(class_path, image_name)
                image = cv2.imread(image_path)
                # convert image to greyscale and 64x64
                image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
                image = cv2.resize(image, (image_size, image_size))
                images.append(image)
                labels.append(class_index)

images = np.array(images)
labels = np.array(labels)


## Sanity Check
Quick check on the shapes of the image and label arrays, as well as the classes involved, to ensure that our data extraction and preparation was successful.

In [ ]:
print((images.shape), (labels.shape))
print(class_names)

## Data Showcase
A look at a subset of the data being used in this analysis (along with their class labels).

In [ ]:
num_images_to_display = 3

for _ in range(num_images_to_display):
  index = np.random.randint(0, len(images) - 1)
  image = images[index]
  label = labels[index]
  class_name = class_names[label]

  plt.figure()
  plt.imshow(image, cmap='gray')
  plt.title(f"Class: {class_name}")
  plt.axis('off')
  plt.show()

## Splitting Train and Test Data
The training and test datasets are split and their shapes displayed. A set seed is used for reproducibility purposes.

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size = 0.15, random_state=42)

print(f"Train: {x_train.shape}")
print(f"Test: {x_test.shape}")

## Normalisation
Only a single colour channel is used (greyscale), so its value just needs to be scaled from 0 - 255 to 0 - 1.

In [ ]:
x_train = x_train/255
x_test = x_test/255

## Baseline Model
A simple CNN structure is used to gauge the effectiveness of more complex models.

In [ ]:
# Constructing Model
baseline_model = Sequential()

baseline_model.add(Input(shape=(64, 64, 1)))
baseline_model.add(Conv2D(32, (3, 3), activation='relu'))
baseline_model.add(MaxPooling2D((2, 2)))
baseline_model.add(Flatten())
baseline_model.add(Dense(64, activation='relu'))
baseline_model.add(Dense(4, activation='softmax'))

# Training
opt = optimizers.Adam(learning_rate = 0.001)
baseline_model.compile(optimizer = opt, loss = 'sparse_categorical_crossentropy', metrics = (['accuracy']))
hist = baseline_model.fit(x_train, y_train, epochs = 30, batch_size = 32, validation_split = 0.2)

# Calculating Accuracies
y_train_pred = baseline_model.predict(x_train)
y_train_pred_classes = np.argmax(y_train_pred, axis=1)

y_pred = baseline_model.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)

train_accuracy = accuracy_score(y_train, y_train_pred_classes)
test_accuracy = accuracy_score(y_test, y_pred_classes)

print(f"Training Accuracy: {train_accuracy*100:.2f}%")
print(f"Testing Accuracy: {test_accuracy*100:.2f}%")

## The Final CNN
The CNN model is created using the following architecture:
- An **input** layer of size 64x64x1 (i.e. the image resolution).
- 3x **convolutional/maxpooling** layers using *relu* activation.
- A **flatten** layer used to transition to dense layers.
- A single **dense** layer.
- An **output** layer which produces a result via *softmax* activation.
- Scattered **dropout** layers throughout the model to prevent overfitting.


In [ ]:
K.clear_session()

model = Sequential()

model.add(Input(shape=(64, 64, 1)))

model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.1))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.35))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(4, activation='softmax'))

model.summary()

## Model Compilation and Training
The model is optimised using the widely-used Adam optimiser - which depends only the learning rate. This is a multi-class classification problem so categorical crossentropy is used as the loss function. Performance is measured using the accuracy metric. Training is done using min-batches over a number of epochs.

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size = 0.2, random_state=42)

opt = optimizers.Adam(learning_rate = 0.001)
model.compile(optimizer = opt, loss = 'sparse_categorical_crossentropy', metrics = (['accuracy']))
hist = model.fit(x_train, y_train, epochs = 30, batch_size = 64, validation_data = (x_val, y_val))

## Model Predictions
Here one instance of a model's predictive capabilities are shown for a randomly selected test data image. The image's predicted and actual classes are both shown.

In [ ]:
y_hat = model.predict(x_test)

x = np.random.randint(0, len(x_test))
pred_class = np.argmax(y_hat[x])
pred_class = class_names[pred_class]
acu = class_names[y_test[x]]


t = f'Pred -> {pred_class}'
p = f'Actual -> {acu}'

plt.figure()
plt.imshow(x_test[x], cmap="gray")
plt.title(t)
plt.xlabel(p)

## Model Performance
Plotting training and validation accuracies and losses as a function of epoch number to show that the model doesn't overfit.

In [ ]:
# accuracy
train_acc = hist.history['accuracy']
val_acc = hist.history['val_accuracy']

epochs = range(1, len(train_acc) + 1)

plt.figure(figsize=(8, 6))
plt.plot(epochs, train_acc, 'b.-', label='Training')
plt.plot(epochs, val_acc, 'r.-', label='Validation')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# loss
train_loss = hist.history['loss']
val_loss = hist.history['val_loss']

epochs = range(1, len(train_loss) + 1)

plt.figure(figsize=(8, 6))
plt.plot(epochs, train_loss, 'b.-', label='Training')
plt.plot(epochs, val_loss, 'r.-', label='Validation')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

Here the training and testing accuracies and F1 scores for the CNN model are calculated and displayed.

In [ ]:
y_train_pred = model.predict(x_train)
y_train_pred_classes = np.argmax(y_train_pred, axis=1)

y_val_pred = model.predict(x_val)
y_val_pred_classes = np.argmax(y_val_pred, axis=1)

y_pred = model.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)

# Accuracies
train_accuracy = accuracy_score(y_train, y_train_pred_classes)
val_accuracy = accuracy_score(y_val, y_val_pred_classes)
test_accuracy = accuracy_score(y_test, y_pred_classes)

# F1 Scores
train_f1 = f1_score(y_train, y_train_pred_classes, average='weighted')
val_f1 = f1_score(y_val, y_val_pred_classes, average='weighted')
test_f1 = f1_score(y_test, y_pred_classes, average='weighted')

# Outputting results
print("Training")
print(f"\tAccuracy: {train_accuracy*100:.2f}%")
print(f"\tF1 Score: {train_f1:.3f}")

print("\nValidation")
print(f"\tAccuracy: {val_accuracy*100:.2f}%")
print(f"\tF1 Score: {val_f1:.3f}")

print("\nTesting")
print(f"\tAccuracy: {test_accuracy*100:.2f}%")
print(f"\tF1 Score: {test_f1:.3f}")
